# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # skip header
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance in the local machine 
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify_analytics
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify_analytics')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
- **Definition:** Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.
- **Requirement:** We need to filter rows based on sessionId and itemInSession.
- **Select statement:** "SELECT artist_name, song_title, songs_length FROM songs_by_session WHERE session_id = 338 AND item_in_session = 4"
- **Primary key:** (session_id, item_in_session), uniquely identifies each row and suitable for WHERE clause for the given query. Partition key 'session_id' should roughly provide equal distribution of rows across the cluster and all items for a session resides on the same physical node. 

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session \
         (session_id bigint, \
          item_in_session int, \
          artist_name text, \
          song_title text, \
          songs_length decimal, \
          PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert into the table row by row
        query = "INSERT INTO songs_by_session (session_id, item_in_session, artist_name, song_title, songs_length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist_name, song_title, songs_length FROM songs_by_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Retrieve the query results into pandas dataframe
df_q1 = pd.DataFrame(columns=['artist_name', 'song_title', 'songs_length'])
for idx, row in enumerate(rows):
    df_q1.loc[idx] = [row.artist_name, row.song_title, row.songs_length]

# Display the results
df_q1.head()

,artist_name,song_title,songs_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


## Query 2
- **Definition:** Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182.
- **Requirement:** We need to filter rows based on userId and sessionId and sort (cluster) based on itemInSession.
- **Select statement:** "SELECT artist_name, song_title, user_firstname, user_lastname FROM songs_by_user WHERE user_id = 10 AND session_id = 182"
- **Primary key:** ((user_id, session_id), item_in_session), uniquely identifies each row and suitable for WHERE clause for the given query. Composite partition key (user_id, session_id) together with 'item_in_session' clustering column sorts the songs per user and session.

In [11]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user \
         (user_id bigint, \
          session_id bigint, \
          item_in_session int, \
          artist_name text, \
          song_title text, \
          user_firstname text, \
          user_lastname text, \
          PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert into the table row by row
        query = "INSERT INTO songs_by_user (user_id, session_id, item_in_session, artist_name, song_title, user_firstname, user_lastname) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [13]:
query = "SELECT artist_name, song_title, user_firstname, user_lastname FROM songs_by_user WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Retrieve the query results into pandas dataframe
df_q2 = pd.DataFrame(columns=['artist_name', 'song_title', 'user_firstname', 'user_lastname'])
for idx, row in enumerate(rows):
    df_q2.loc[idx] = [row.artist_name, row.song_title, row.user_firstname, row.user_lastname]

# Display the results
df_q2.head()

,artist_name,song_title,user_firstname,user_lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3
- **Definition:** Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own.
- **Requirement:** We need to filter rows based on a song and return all users who listened to it.
- **Select statement:** "SELECT user_firstname, user_lastname, user_id FROM users_by_song WHERE song_title = 'All Hands Against His Own'"
- **Primary key:** (song_title , user_id), each user-song pair is uniquely identified. 'user_id' should be used as clustring column because first & last name do not uniquely represents users.

In [14]:
query = "CREATE TABLE IF NOT EXISTS users_by_song \
         (song_title text, \
          user_id bigint, \
          user_firstname text, \
          user_lastname text, \
          PRIMARY KEY (song_title , user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert into the table row by row
        query = "INSERT INTO users_by_song (song_title, user_id, user_firstname, user_lastname) "
        query = query + "VALUES (%s,%s,%s,%s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [16]:
query = "SELECT user_id, user_firstname, user_lastname FROM users_by_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Retrieve the query results into pandas dataframe
df_q3 = pd.DataFrame(columns=['user_firstname', 'user_lastname', 'user_id'])
for idx, row in enumerate(rows):
    df_q3.loc[idx] = [row.user_firstname, row.user_lastname, row.user_id]

# Display the results
df_q3.head()

,user_firstname,user_lastname,user_id
0,Jacqueline,Lynch,29
1,Tegan,Levine,80
2,Sara,Johnson,95


### Drop the tables before closing out the sessions

In [17]:
table_name = ["songs_by_session", "songs_by_user", "users_by_song"]
for table in table_name:
    try:
        query = "DROP TABLE IF EXISTS {}".format(table)
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()